In [1]:
class SVR_:
    
    def __init__(self, C = 0.1, epsilon = 0.01, kernel = "linear", error = 1e-5, **kernel_param):
        import numpy as np
        from cvxopt import matrix, solvers, sparse
        from sklearn.metrics.pairwise import pairwise_kernels
        from sklearn.utils import check_X_y, check_array 
        self.sparse = sparse
        self.matrix = matrix
        self.solvers = solvers
        self.C = C
        self.epsilon = epsilon
        self.kernel = kernel
        self.error = error
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        self.check_X_y = check_X_y
        self.check_array = check_array
        
    def fit(self, X, y):
        X, y = self.check_X_y(X, y)
        # hyperparameters
        C = self.C 
        epsilon =  self.epsilon
        
        kernel = self.kernel
        error = self.error
        pairwise_kernels = self.pairwise_kernels
        
        sparse = self.sparse 
        matrix = self.matrix 
        solvers = self.solvers 
        
        # Useful parameters
        ydim = y.shape[0]
        onev = np.ones((ydim,1))
        x0 = np.random.rand(ydim)
        
        # Prematrices for the optimizer
        K = pairwise_kernels(X, X, metric = kernel, **self.kernel_param)
        A = onev.T
        b = 0.0
        G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
        h_ = np.concatenate((C*np.ones(ydim), C*np.ones(ydim))); 
        h = h_.reshape(-1, 1)

        # Matrices for the optimizer
        A = matrix(A)
        b = matrix(b)
        G = sparse(matrix(G))
        h = matrix(h)
        Ev = (epsilon*onev.T)
        
        # functions for the optimizer
        def obj_func(x):
            return 0.5* x.T @ K @ x + Ev @ (np.abs(x)) - y.T @ x
        def obj_grad(x):
            return x.T @ K + Ev @ (x/np.abs(x)) - y.T
        
        def F(x = None, z = None):
            if x is None: return 0, matrix(x0)
            # objective dunction
            val = matrix(obj_func(x))
            # obj. func. gradient
            Df = matrix(obj_grad(x))
            if z is None: return val, Df
            # hessian
            H = matrix(z * K)
            return val, Df, H
        
        # Solver
        solvers.options['show_progress'] = False
        sol = solvers.cp(F=F, G=G, h=h, A=A, b=b)
        
        # support vectors
        beta_1 = np.array(sol['x'])
        indx = abs(beta_1) > error
        beta_sv = beta_1[indx]
        x_sv = X[indx[:,0],:]
        y_sv = y[indx[:,0]]
        
        # get w_phi and b
        k_sv = pairwise_kernels(x_sv, x_sv, metric = kernel, **self.kernel_param)
        cons = np.where(beta_sv >= 0, -epsilon, epsilon)
        
        w_phi = beta_sv @ k_sv
        b = np.mean((cons - w_phi + y_sv)); self.b = b
        self.beta_sv = beta_sv; self.x_sv = x_sv
        return self
        
    def predict(self, X_):
        X_ = self.check_array(X_)
        k_test = self.pairwise_kernels(self.x_sv, X_, metric = self.kernel, **self.kernel_param)
        w_phi_test = self.beta_sv @ k_test
        predict = w_phi_test + self.b
        return predict
    
    def coef_(self):
        """--Returns--
                - dual support vectors
                - primal support vectors
                - intercept"""
        return self.beta_sv, self.x_sv, self.b


In [ ]:
class SVR_cvxpy:
    
    def __init__(self, C = 0.1, epsilon = 0.01, kernel = "linear", error = 1e-5, **kernel_param):
        import cvxpy as cp
        import numpy as np
        from sklearn.metrics.pairwise import pairwise_kernels
        from sklearn.utils import check_X_y, check_array 
        self.cp = cp
        self.C = C
        self.epsilon = epsilon
        self.kernel = kernel
        self.error = error
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        self.check_X_y = check_X_y
        self.check_array = check_array
        
    def fit(self, X, y):
        X, y = self.check_X_y(X, y)
        # hyperparameters
        C = self.C 
        epsilon =  self.epsilon
        kernel = self.kernel
        error = self.error
        pairwise_kernels = self.pairwise_kernels
        cp = self.cp
        # Useful parameters
        ydim = y.shape[0]
        onev = np.ones((ydim,1))
        
        # Matrices for the optimizer
        K = pairwise_kernels(X, X, metric = kernel, **self.kernel_param)
        A = onev.T
        b = 0
        G = np.concatenate((np.identity(ydim), -np.identity(ydim)))
        h_ = np.concatenate((C*np.ones(ydim), C*np.ones(ydim))); 
        h = h_.reshape(-1, 1)
        
        # loss function and constraints
        beta = cp.Variable((ydim,1))
        Ev = (epsilon*onev)
        objective = cp.Minimize((1/2)*cp.quad_form(beta, K) + Ev.T @ cp.abs(beta) - y.T @ beta)
        constraints = [A @ beta == b, G @ beta <= h]
        
        # Solver and solution
        prob = cp.Problem(objective,constraints)
        result = prob.solve(solver='ECOS')
        
        # support vectors
        beta_1 = np.array(beta.value)
        indx = abs(beta_1) > error
        beta_sv = beta_1[indx]
        x_sv = X[indx[:,0],:]
        y_sv = y[indx[:,0]]
        
        # get w_phi and b
        k_sv = pairwise_kernels(x_sv, x_sv, metric = kernel, **self.kernel_param)
        cons = np.where(beta_sv >= 0, -epsilon, epsilon)
        
        w_phi = beta_sv @ k_sv
        b = np.mean((cons - w_phi + y_sv)); self.b = b
        self.beta_sv = beta_sv; self.x_sv = x_sv
        return self
        
    def predict(self, X_):
        X_ = self.check_array(X_)
        k_test = self.pairwise_kernels(self.x_sv, X_, metric = self.kernel, **self.kernel_param)
        w_phi_test = self.beta_sv @ k_test
        predict = w_phi_test + self.b
        return predict
    
    def coef_(self):
        """--Returns--
                - dual support vectors
                - primal support vectors
                - intercept"""
        return self.beta_sv, self.x_sv, self.b